In [1]:
import math
from creme import compose
from creme import feature_extraction
from creme import naive_bayes
import creme

In [2]:
docs = [ ('Chinse Beijing Chinese', 'yes'), ('Chinese Chincese Shangai', 'yes'),('Chinese Macao', 'yes'), ('Tokyo Japan Chinese', 'no')]

In [3]:
docs

[('Chinse Beijing Chinese', 'yes'),
 ('Chinese Chincese Shangai', 'yes'),
 ('Chinese Macao', 'yes'),
 ('Tokyo Japan Chinese', 'no')]

In [4]:
model = compose.Pipeline(
    ('tokenize', feature_extraction.BagOfWords(lowercase=False)),
    ('nb', naive_bayes.MultinomialNB(alpha=1))
)

In [5]:
#This is just and example

corpus = [
    'This is the first document.'
    'This document is the second docmunent.'
    'And this is the thirt one.'
    'Is this the first document?'
]

bow = creme.feature_extraction.TFIDF()
for sentence in corpus:
    print(bow.transform_one(sentence))

{'this': 0.49236596391733095, 'is': 0.49236596391733095, 'the': 0.49236596391733095, 'first': 0.24618298195866548, 'document': 0.36927447293799814, 'second': 0.12309149097933274, 'docmunent': 0.12309149097933274, 'and': 0.12309149097933274, 'thirt': 0.12309149097933274, 'one': 0.12309149097933274}


In [6]:
%%time
for sentence, label in docs:
    model = model.fit_one(sentence, label)

CPU times: total: 0 ns
Wall time: 0 ns


In [7]:
new_unseen = "India USA Tokyo"


In [8]:
model.predict_one(new_unseen)

'no'

In [9]:
#Training for the new data 
model.fit_one('Tokyo India USA', 'no')

Pipeline (
  BagOfWords (
    on=None
    strip_accents=True
    lowercase=False
    preprocessor=None
    tokenizer=<built-in method findall of re.Pattern object at 0x000002D940CE9000>
    ngram_range=(1, 1)
  ),
  MultinomialNB (
    alpha=1
  )
)

<b> Taining for the new data and new category:

In [10]:
model.fit_one('Taiwan Honkong Tibet', 'may be')

Pipeline (
  BagOfWords (
    on=None
    strip_accents=True
    lowercase=False
    preprocessor=None
    tokenizer=<built-in method findall of re.Pattern object at 0x000002D940CE9000>
    ngram_range=(1, 1)
  ),
  MultinomialNB (
    alpha=1
  )
)

In [11]:
model.predict_one('Taiwan Canada')

'may be'

In [12]:
!pip install pandas

In [14]:
#NLP
import pandas as pd

messege = pd.read_csv('smsspamcollection/SMSSpamCollection', sep='\t', names=["label", "message"])

In [15]:
messege.shape

(5572, 2)

In [16]:
import pandas as pd
from sklearn.model_selection import train_test_split

messege_train, messege_test = train_test_split(messege)

In [17]:
messege_train

,label,message
1708,ham,Was doing my test earlier. I appreciate you. W...
2410,ham,Aww that's the first time u said u missed me w...
1326,ham,Yeah jay's sort of a fucking retard
3710,ham,Ok.ok ok..then..whats ur todays plan
5421,ham,"Hi elaine, is today's meeting confirmed?"
...,...,...
1314,ham,How abt making some of the pics bigger?
3042,ham,Aight what time you want me to come up?
2546,ham,So are you guys asking that i get that slipper...
5018,spam,Dear 0776xxxxxxx U've been invited to XCHAT. T...


In [18]:
#Converting Dataframe to tuples

messege_train = messege_train.to_records(index= False)
messege_test = messege_test.to_records(index=False)

In [19]:
messege_train

rec.array([('ham', 'Was doing my test earlier. I appreciate you. Will call you tomorrow.'),
           ('ham', "Aww that's the first time u said u missed me without asking if I missed u first. You DO love me! :)"),
           ('ham', "Yeah jay's sort of a fucking retard"), ...,
           ('ham', 'So are you guys asking that i get that slippers again or its gone with last year'),
           ('spam', "Dear 0776xxxxxxx U've been invited to XCHAT. This is our final attempt to contact u! Txt CHAT to 86688 150p/MsgrcvdHG/Suite342/2Lands/Row/W1J6HL LDN 18yrs"),
           ('spam', 'Eerie Nokia tones 4u, rply TONE TITLE to 8007 eg TONE DRACULA to 8007 Titles: GHOST, ADDAMSFA, MUNSTERS, EXORCIST, TWILIGHT www.getzed.co.uk POBox36504W45WQ 150p ')],
          dtype=[('label', 'O'), ('message', 'O')])

In [20]:
# Creating the pipeline
# 1st function is creating the TFIDF
# 2nd function is the naive bayes predictor
import math
from creme import compose
from creme import feature_extraction
from creme import naive_bayes
import creme
model = compose.Pipeline(
    ('tokenize', feature_extraction.TFIDF(lowercase=False)),
    ('nb', naive_bayes.MultinomialNB(alpha=1))
)

In [21]:
from creme import metrics
metric=metrics.Accuracy()
# Training the model row by row
for label,sentence in messege_train:
    model = model.fit_one(sentence, label)
    y_pred = model.predict_one(sentence)
    metric = metric.update(label, y_pred)


In [22]:
### Training Data Accuracy
metric

Accuracy: 95.79%

In [23]:
### test Data Accuracy
test_metric=metrics.Accuracy()
for label,sentence in messege_test:
    y_pred = model.predict_one(sentence)
    test_metric = metric.update(label, y_pred)

In [24]:
### test Metric
test_metric

Accuracy: 96.09%

#### Creme with Logistic Regression

In [13]:
from creme import compose
from creme import datasets
from creme import linear_model
from creme import metrics
from creme import preprocessing

X_y = datasets.Phishing()

model = compose.Pipeline(
   preprocessing.StandardScaler(),
 linear_model.LogisticRegression()
)
metric = metrics.Accuracy()


for x, y in X_y:
    y_pred = model.predict_one(x)      # make a prediction
    metric = metric.update(y, y_pred)  # update the metric
    model = model.fit_one(x, y)        # make the model learn

metric

Accuracy: 89.20%